In [ ]:
import pandas as pd
import numpy as np

def sigmoid(activation):
    return 1.0 / (1.0 + np.exp(-activation))

def back_prop(train_X,W1,W2,hidden_layer_output,layer_output,actual_output): 
    delta_output = layer_output*(1-layer_output)*(actual_output.T - layer_output)
    delta_hidden = hidden_layer_output*(1-hidden_layer_output)*W2.T.dot(delta_output)
    deltaW2 = lr*(delta_output.dot(hidden_layer_output.T)/n_train) 
    deltaW1 = lr*(delta_hidden.dot(train_X)/n_train) 
    
    return (deltaW1,deltaW2)
    
def train_network(train_X, train_y):
    actual_output = train_y
    n_input = train_X.shape[1]  # the number of columns in the training data
    W1=np.random.random((n_hidden,n_input))
    W2=np.random.random((num_classes,n_hidden ))
    for epoch in range(n_epoch):
        hidden_layer_output = sigmoid(W1.dot(train_X.T))
        layer_output = sigmoid(W2.dot(hidden_layer_output))
        
        (deltaW1,deltaW2)= back_prop(train_X,W1,W2,hidden_layer_output,layer_output,actual_output)
#        print(deltaW1[:5])
        W2 = W2+deltaW2
        W1 = W1+deltaW1
        if epoch%1000 == 0:
            loss = 0.5*np.sum((actual_output.T-layer_output)*(actual_output.T-layer_output));
            print(str.format('Loss in {0}th epoch is {1}',epoch,loss))
        
    return (W1,W2)

def evaluate(test_X,test_y,params):
    actual_test_output = test_y
    (W1,W2) = params
    hidden_layer_output = sigmoid(W1.dot(test_X.T))
    final = sigmoid(W2.dot(hidden_layer_output))
#    print("final = ",final)
    prediction = final.argmax(axis=0)    
#    print("prediction = ",prediction)
    return np.sum(prediction==actual_test_output)/len(actual_test_output)    

def split_dataset(dataset, train_perc=0.8):
    np.random.shuffle(dataset)
    train_index = int(len(dataset)*train_perc) # last index from the dataset array that will go into training
    
    train = dataset[:train_index,:]  
    test = dataset[train_index:,:] 
    return (train, test)

def convert_to_OH(data,num_classes):
    one_hot = np.zeros((len(data),num_classes))
    one_hot[np.arange(len(data)),data] = 1
    return one_hot


#np.random.seed(0)
df = pd.read_csv('diabetes.csv',header = None,dtype=np.float64)
dataset = np.array(df)

#normalize data
min_data = dataset.min(axis = 0) #minimum values across each column
max_data = dataset.max(axis = 0) #maximum values across each column

#normalize all fields except the last column(class)
dataset[:,:-1] = (dataset[:,:-1] - min_data[:-1])/(max_data[:-1] - min_data[:-1])
(train, test) = split_dataset(dataset)
print(train[:5,:-1])
n_train = len(train)
n_test = len(test)

num_classes = len(np.unique(dataset[:,-1]))
train_one_hot = convert_to_OH(train[:,-1].astype(int),num_classes)

n_hidden = 15
lr = 0.8
n_epoch =15000

params = train_network(train[:,:-1],train_one_hot) 
accuracy = evaluate(test[:,:-1],test[:,-1],params)*100
print('Mean Accuracy: %.3f%%' % accuracy)
